In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
#=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
website_url = requests.get("https://es.wikipedia.org/wiki/Anexo:Localidades_de_Bogot%C3%A1").text

In [ ]:
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

In [ ]:
My_table = soup.find('table',{'class':'sortable wikitable'})
My_table

In [ ]:
links = My_table.findAll('a')
links

In [ ]:
Localidades = []
for link in links:
    Localidades.append(link.get('title'))
    
print(Localidades)

del Localidades[0:2]

Localidades

In [ ]:
df = pd.DataFrame()
df['Localidades'] = Localidades
df

In [ ]:
def get_coords_local(localidad, output_as='center'):
    """
    get the bounding box of a locality in WGS84 given its name

    Parameters
    ----------
    localidad : str
        name of the country in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]

    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             localidad+', Bogota, Bogota Capital District',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

In [ ]:
df2 = df.copy()

latitudeCln = []
longitudeCln = []
for index, row in df2.iterrows():
    print(row[0])
    lat, long = get_coords_local(localidad=row[0], output_as='center')
    latitudeCln.append(lat)
    longitudeCln.append(long)

df2['Latitude'] = latitudeCln
df2['Longitude'] = longitudeCln

df2.shape

In [ ]:
df2

In [ ]:
address = 'Bogotá, Colombia'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Bogotá are {}, {}.'.format(latitude, longitude))

In [ ]:
# make a copy of the dataframe to get it simply back if needed
df3 = df2.copy()
df3

In [ ]:
# create map of Bogotá using latitude and longitude values
map_bogota = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(df3['Latitude'], df3['Longitude'], df3['Localidades']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bogota)  
    
map_bogota

In [ ]:
# function to repeat the exploring process to all the neighborhoods in Toronto
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Localidad', 
                  'Localidad Latitude', 
                  'Localidad Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [ ]:
limit = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [ ]:
# Use category id 4bf58dd8d48988d16c941735 to only get the burger joints
bogota_venues_burger = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16c941735')
bogota_venues_burger.head()

In [ ]:
bogota_venues_burger.shape

In [ ]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Localidad'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [ ]:
map_bogota_burger = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_burger, 'red', map_bogota_burger)
map_bogota_burger

In [ ]:
bogota_venues_highschools = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d13d941735')
bogota_venues_highschools.head()

In [ ]:
bogota_venues_highschools.shape

In [ ]:
map_bogota_highschools = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_highschools, 'green', map_bogota_highschools)
map_bogota_highschools

In [ ]:
bogota_venues_uni = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ae941735')
bogota_venues_uni.head()

In [ ]:
bogota_venues_uni.shape

In [ ]:
map_bogota_uni = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_uni, 'gold', map_bogota_uni)
map_bogota_uni

In [ ]:
bogota_venues_office = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4d4b7105d754a06375d81259')
bogota_venues_office.head()

In [ ]:
bogota_venues_office.shape

In [ ]:
map_bogota_office = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_office, 'fuchsia', map_bogota_office)
map_bogota_office

In [ ]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Localidad').count()
    
    for n in startDf['Localidad']:
        try:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = 0

In [ ]:
df_data = df3.copy()
df_data.rename(columns={'Localidades':'Localidad'}, inplace=True)
addColumn(df_data, 'Burger', bogota_venues_burger)
addColumn(df_data, 'High Schools', bogota_venues_highschools)
addColumn(df_data, 'Universities', bogota_venues_uni)
addColumn(df_data, 'Offices', bogota_venues_office)
df_data

In [ ]:
# negative weight, because Jeronimo wants to open a burger joint and thus wants to avoid concurrence as much as possible
weight_burger = -1

# positive weight, because high school students are good customers
weight_schools = 1

# positive weight, because uni students are good customers
weight_uni = 1.5

# positive weight because employees are even better customers
weight_offices = 2

In [ ]:
df_weighted = df_data[['Localidad']].copy()

In [ ]:
df_weighted['Score'] = df_data['Burger'] * weight_burger + df_data['High Schools'] * weight_schools + df_data['Universities'] * weight_uni + df_data['Offices'] * weight_offices
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted

In [ ]:
map_bog_result = folium.Map(location=[latitude, longitude], zoom_start=15)

bog_win = df3[df3['Localidades'] == 'La Candelaria']

for lat, lng, local in zip(bog_win['Latitude'], bog_win['Longitude'], bog_win['Localidades']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_bog_result) 

addToMap(bogota_venues_burger[bogota_venues_burger['Localidad'] == 'La Candelaria'], 'red', map_bog_result)
addToMap(bogota_venues_highschools[bogota_venues_highschools['Localidad'] == 'La Candelaria'], 'green', map_bog_result)
addToMap(bogota_venues_uni[bogota_venues_uni['Localidad'] == 'La Candelaria'], 'gold', map_bog_result)
addToMap(bogota_venues_office[bogota_venues_office['Localidad'] == 'La Candelaria'], 'fuchsia', map_bog_result)

map_bog_result